In [8]:
import os
import pandas as pd 
import math
import nibabel as nb
import numpy as np
#pd.set_option('display.max_rows', 10)

First, load in all the subjects .nii files. Then, import the CSV file that specifies the timestamps where internal monologue is present during each video. For each video, the subject's neural timeseries file will be segmented out into 25 clips. This means that altogether, each subject will have 50 clips. 

In [42]:
inarr_data = '/Volumes/LT/phd/inarr/'

sublist = [x for x in os.listdir(inarr_data) if 'sub' in x]

timestamps = pd.read_csv('/Users/lindseytepfer/Documents/phd/inarr/monologue_timestamps.csv')

v1_subs = ['sub-1','sub-10','sub-13','sub-14','sub-17','sub-18','sub-19',
'sub-20','sub-21','sub-24','sub-25','sub-26','sub-27']
v2_subs = ['sub-2','sub-3','sub-4','sub-5','sub-6','sub-7','sub-8','sub-9',
'sub-11','sub-12','sub-15','sub-16','sub-22','sub-23', 'sub-28']

This function takes in each .nii image and the subsetted dataframe, and uses the timestamps inside of the dataframe to determine where the nii files must be trimmed. Because we're dealing with brain data, we shift the timecourse by 6 seconds to account for the hemodynamic response function. 

In [37]:
def trim_image(brain, df, snum, movie):

    for i in range(df.index.min(),df.index.max()+1):

        start = math.floor((df.loc[i, "start"] / 2) + 6)
        stop = math.ceil((df.loc[i, "stop"] /2) + 6)
        type = df.loc[i, "type"]
        version_silenced = df.loc[i, "version_silenced"]

        if snum in v1_subs:
            version = 1
        elif snum in v2_subs:
            version = 2

        if i%2 == 1: #this means its an IM segment
            if version_silenced == version:
                version = str(version)+"_silenced"

        if i != df.index[-1]:
            sub_trimmed = brain.slicer[:,:,:,start:stop]
        else:
            sub_trimmed = brain.slicer[:,:,:,start:]

        nb.save(sub_trimmed,inarr_data+snum+"/segmented_files/"+snum+"_"+movie+"_clip-"+str(i)+"_"+type+"_version-"+str(version)+".nii.gz")
    
    return (print("trimming complete."))

In [ ]:
for sub in sublist:
    print(sub)
    sub_files = [x for x in os.listdir(inarr_data+sub) if '.nii.gz' in x]

    for f in sub_files:
        sub_brain = nb.load(inarr_data+sub+os.sep+f)

        if 'physical' in f:
            phys_df = timestamps[(timestamps.movie == 'physical')].copy()
            trim_image(sub_brain, phys_df, sub, 'physical')

        elif 'stutterer' in f:
            stut_df = timestamps[(timestamps.movie == 'stutterer')].copy().reset_index()
            trim_image(sub_brain, stut_df, sub, 'stutterer')



sub-16
trimming complete.
trimming complete.
sub-29
sub-2
trimming complete.
trimming complete.
sub-5
trimming complete.
trimming complete.
sub-4
trimming complete.
trimming complete.
sub-3
trimming complete.
trimming complete.
